## Lab| Customer Analysis Case Study| Modeling

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import QuantileTransformer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PowerTransformer
from sklearn.metrics import r2_score
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
from scipy.stats import boxcox
pd.options.display.max_rows = 50
pd.options.display.max_columns = 999
import warnings
warnings.filterwarnings('ignore')

In [2]:
customer_df=pd.read_csv("Data/clean_data.csv" )

### Xy Split

In [3]:
X=customer_df.drop('total_claim_amount', axis=1)
y=customer_df.total_claim_amount

In [4]:
X

,Unnamed: 0,state,customer_lifetime_value,response,coverage,employmentstatus,gender,income,location_code,marital_status,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,vehicle_class,vehicle_size
0,0,Arizona,4809.0,No,Basic,Employed,M,48029,Suburban,Married,61,7.0,52,0.0,3,Corporate Auto,Corporate,Offer3,Agent,Four-Door Car,Medsize
1,2,Washington,14948.0,No,Basic,Employed,M,22139,Suburban,Single,100,34.0,31,0.0,2,Personal Auto,Personal,Offer3,Call Center,Four-Door Car,Medsize
2,6,California,5035.0,No,Basic,Employed,F,37405,Urban,Married,63,8.0,99,3.0,3,Corporate Auto,Corporate,Offer2,Branch,Four-Door Car,Medsize
3,7,California,4956.0,No,Basic,Employed,M,87197,Urban,Single,63,35.0,45,0.0,3,Personal Auto,Personal,Offer2,Branch,Two-Door Car,Medsize
4,8,California,5990.0,Yes,Premium,Employed,M,66839,Suburban,Single,154,33.0,24,0.0,1,Personal Auto,Personal,Offer2,Branch,Sports Car,Medsize
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5865,10901,Washington,4548.0,No,Premium,Employed,M,33531,Suburban,Married,116,11.0,15,0.0,1,Personal Auto,Personal,Offer4,Agent,Two-Door Car,Medsize
5866,10902,California,3579.0,No,Extended,Employed,F,28304,Suburban,Married,91,10.0,30,2.0,1,Personal Auto,Personal,Offer2,Agent,Four-Door Car,Medsize
5867,10903,Arizona,2772.0,No,Basic,Employed,M,59855,Suburban,Married,74,30.0,82,4.0,1,Personal Auto,Personal,Offer2,Branch,Two-Door Car,Medsize
5868,10906,Oregon,5259.0,No,Basic,Employed,F,61146,Urban,Married,65,7.0,68,0.0,3,Personal Auto,Personal,Offer2,Branch,Four-Door Car,Medsize


In [5]:
numericals_features =X._get_numeric_data()
# numericals =data.select_dtypes(np.number)
numericals_features.drop('Unnamed: 0', axis=1, inplace=True)

In [6]:
numericals_features.describe()

,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies
count,5870.000000,5870.000000,5870.000000,5870.000000,5870.000000,5870.000000,5870.000000
mean,6243.174446,50637.092845,86.365588,14.972402,47.726746,0.390119,2.040204
std,3172.058923,24388.055540,21.965941,10.029939,27.900584,0.919732,0.898543
min,2120.000000,10037.000000,61.000000,0.000000,0.000000,0.000000,1.000000
25%,3746.250000,28520.000000,68.000000,6.000000,24.000000,0.000000,1.000000
50%,5452.000000,48029.000000,79.000000,14.000000,47.000000,0.000000,2.000000
75%,8208.000000,70411.500000,104.000000,23.000000,71.000000,0.000000,3.000000
max,16532.000000,99981.000000,165.000000,35.000000,99.000000,5.000000,3.000000


In [7]:
categoricals_features = X.select_dtypes(object)
categoricals_features

,state,response,coverage,employmentstatus,gender,location_code,marital_status,policy_type,policy,renew_offer_type,sales_channel,vehicle_class,vehicle_size
0,Arizona,No,Basic,Employed,M,Suburban,Married,Corporate Auto,Corporate,Offer3,Agent,Four-Door Car,Medsize
1,Washington,No,Basic,Employed,M,Suburban,Single,Personal Auto,Personal,Offer3,Call Center,Four-Door Car,Medsize
2,California,No,Basic,Employed,F,Urban,Married,Corporate Auto,Corporate,Offer2,Branch,Four-Door Car,Medsize
3,California,No,Basic,Employed,M,Urban,Single,Personal Auto,Personal,Offer2,Branch,Two-Door Car,Medsize
4,California,Yes,Premium,Employed,M,Suburban,Single,Personal Auto,Personal,Offer2,Branch,Sports Car,Medsize
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5865,Washington,No,Premium,Employed,M,Suburban,Married,Personal Auto,Personal,Offer4,Agent,Two-Door Car,Medsize
5866,California,No,Extended,Employed,F,Suburban,Married,Personal Auto,Personal,Offer2,Agent,Four-Door Car,Medsize
5867,Arizona,No,Basic,Employed,M,Suburban,Married,Personal Auto,Personal,Offer2,Branch,Two-Door Car,Medsize
5868,Oregon,No,Basic,Employed,F,Urban,Married,Personal Auto,Personal,Offer2,Branch,Four-Door Car,Medsize


### Categorical Variables Transformation and reunion all_features

In [8]:
# get dummies pandas
categoricals_features=pd.get_dummies(categoricals_features, drop_first=True)
categoricals_features

,state_California,state_Nevada,state_Oregon,state_Washington,response_Yes,coverage_Extended,coverage_Premium,employmentstatus_Employed,employmentstatus_Medical Leave,employmentstatus_Retired,gender_M,location_code_Suburban,location_code_Urban,marital_status_Married,marital_status_Single,policy_type_Personal Auto,policy_type_Special Auto,policy_Personal,policy_Special,renew_offer_type_Offer2,renew_offer_type_Offer3,renew_offer_type_Offer4,sales_channel_Branch,sales_channel_Call Center,sales_channel_Web,vehicle_class_Sports Car,vehicle_class_Two-Door Car,vehicle_size_Medsize,vehicle_size_Small
0,0,0,0,0,0,0,0,1,0,0,1,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0
1,0,0,0,1,0,0,0,1,0,0,1,1,0,0,1,1,0,1,0,0,1,0,0,1,0,0,0,1,0
2,1,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0
3,1,0,0,0,0,0,0,1,0,0,1,0,1,0,1,1,0,1,0,1,0,0,1,0,0,0,1,1,0
4,1,0,0,0,1,0,1,1,0,0,1,1,0,0,1,1,0,1,0,1,0,0,1,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5865,0,0,0,1,0,0,1,1,0,0,1,1,0,1,0,1,0,1,0,0,0,1,0,0,0,0,1,1,0
5866,1,0,0,0,0,1,0,1,0,0,0,1,0,1,0,1,0,1,0,1,0,0,0,0,0,0,0,1,0
5867,0,0,0,0,0,0,0,1,0,0,1,1,0,1,0,1,0,1,0,1,0,0,1,0,0,0,1,1,0
5868,0,0,1,0,0,0,0,1,0,0,0,0,1,1,0,1,0,1,0,1,0,0,1,0,0,0,0,1,0


In [9]:
all_features=pd.concat([numericals_features, categoricals_features], axis=1) # concat numerical and categorical transformations
all_features

,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,state_California,state_Nevada,state_Oregon,state_Washington,response_Yes,coverage_Extended,coverage_Premium,employmentstatus_Employed,employmentstatus_Medical Leave,employmentstatus_Retired,gender_M,location_code_Suburban,location_code_Urban,marital_status_Married,marital_status_Single,policy_type_Personal Auto,policy_type_Special Auto,policy_Personal,policy_Special,renew_offer_type_Offer2,renew_offer_type_Offer3,renew_offer_type_Offer4,sales_channel_Branch,sales_channel_Call Center,sales_channel_Web,vehicle_class_Sports Car,vehicle_class_Two-Door Car,vehicle_size_Medsize,vehicle_size_Small
0,4809.0,48029,61,7.0,52,0.0,3,0,0,0,0,0,0,0,1,0,0,1,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0
1,14948.0,22139,100,34.0,31,0.0,2,0,0,0,1,0,0,0,1,0,0,1,1,0,0,1,1,0,1,0,0,1,0,0,1,0,0,0,1,0
2,5035.0,37405,63,8.0,99,3.0,3,1,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0
3,4956.0,87197,63,35.0,45,0.0,3,1,0,0,0,0,0,0,1,0,0,1,0,1,0,1,1,0,1,0,1,0,0,1,0,0,0,1,1,0
4,5990.0,66839,154,33.0,24,0.0,1,1,0,0,0,1,0,1,1,0,0,1,1,0,0,1,1,0,1,0,1,0,0,1,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5865,4548.0,33531,116,11.0,15,0.0,1,0,0,0,1,0,0,1,1,0,0,1,1,0,1,0,1,0,1,0,0,0,1,0,0,0,0,1,1,0
5866,3579.0,28304,91,10.0,30,2.0,1,1,0,0,0,0,1,0,1,0,0,0,1,0,1,0,1,0,1,0,1,0,0,0,0,0,0,0,1,0
5867,2772.0,59855,74,30.0,82,4.0,1,0,0,0,0,0,0,0,1,0,0,1,1,0,1,0,1,0,1,0,1,0,0,1,0,0,0,1,1,0
5868,5259.0,61146,65,7.0,68,0.0,3,0,0,1,0,0,0,0,1,0,0,0,0,1,1,0,1,0,1,0,1,0,0,1,0,0,0,0,1,0


### Train Test Split

In [10]:
X_train, X_test, y_train, y_test=train_test_split(all_features, y, test_size=.20, random_state=42)

In [11]:
X_train

,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,state_California,state_Nevada,state_Oregon,state_Washington,response_Yes,coverage_Extended,coverage_Premium,employmentstatus_Employed,employmentstatus_Medical Leave,employmentstatus_Retired,gender_M,location_code_Suburban,location_code_Urban,marital_status_Married,marital_status_Single,policy_type_Personal Auto,policy_type_Special Auto,policy_Personal,policy_Special,renew_offer_type_Offer2,renew_offer_type_Offer3,renew_offer_type_Offer4,sales_channel_Branch,sales_channel_Call Center,sales_channel_Web,vehicle_class_Sports Car,vehicle_class_Two-Door Car,vehicle_size_Medsize,vehicle_size_Small
2440,7482.0,48992,94,30.0,0,1.0,3,0,0,1,0,0,1,0,1,0,0,0,0,1,1,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0
2029,6746.0,53058,91,16.0,44,4.0,3,1,0,0,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0
5587,2554.0,22538,64,6.0,67,0.0,1,1,0,0,0,0,0,0,1,0,0,0,1,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,1,0
2855,4965.0,98451,62,19.0,57,0.0,3,0,0,0,0,0,0,0,1,0,0,1,1,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,0,1
210,5379.0,24702,68,13.0,63,0.0,3,1,0,0,0,0,0,0,1,0,0,0,0,1,1,0,1,0,1,0,1,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3772,4954.0,45329,65,13.0,7,3.0,3,1,0,0,0,0,0,0,1,0,0,1,1,0,1,0,1,0,1,0,0,0,1,0,1,0,0,1,1,0
5191,14436.0,60880,119,25.0,89,0.0,2,0,0,0,0,1,0,1,1,0,0,1,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,1,1,0
5226,2621.0,26477,66,18.0,37,0.0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,1,0,1,0,1,0,0,1,0,0,0,0,1,0
5390,10657.0,72540,88,7.0,3,0.0,2,0,0,0,1,1,1,0,1,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,1,0,0,1,1,0


#### Standardize

In [12]:
std_scaler=StandardScaler().fit(X_train) 

X_train_scaled=std_scaler.transform(X_train)

In [13]:
X_test_scaled=std_scaler.transform(X_test)

### Modeling

In [14]:
from sklearn.linear_model import Lasso, Ridge, ElasticNet, LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [15]:
model_list = [LinearRegression(), Lasso(alpha=.2), Ridge(), KNeighborsRegressor(n_neighbors=10, p=2)]

In [16]:
def model_application(model_list):
    r2 = dict()
    for model in model_list:
        model.fit(X_train_scaled, y_train)
        y_pred=model.predict(X_test_scaled)
        r2[model] = r2_score(y_test, y_pred)
        print(f"{model}:\n Train: {model.score(X_train_scaled, y_train).round(4)},\n Test: {model.score(X_test_scaled, y_test).round(4)},\n Rsquared: {r2[model].round(4)} \n")

In [17]:
model_application(model_list)

LinearRegression():
 Train: 0.7834,
 Test: 0.797,
 Rsquared: 0.797 

Lasso(alpha=0.2):
 Train: 0.7834,
 Test: 0.7974,
 Rsquared: 0.7974 

Ridge():
 Train: 0.7834,
 Test: 0.797,
 Rsquared: 0.797 

KNeighborsRegressor(n_neighbors=10):
 Train: 0.6234,
 Test: 0.5362,
 Rsquared: 0.5362 



In [18]:
from sklearn.feature_selection import RFE

#all features

model=LinearRegression()
selector = RFE(model, n_features_to_select= 8, step = 1, verbose = 1) # Step is how many features to add or drop everytime
selector.fit(X_train_scaled, y_train)

kept_features = selector.get_support(indices = True) #returns an array of integers corresponding to nonremoved features
kept_features = list(X_train.iloc[:,kept_features].columns)

X_train = selector.transform(X_train_scaled)
X_test  = selector.transform(X_test_scaled)

X_train = pd.DataFrame(X_train, columns=kept_features)
X_test  = pd.DataFrame(X_test, columns=kept_features)

print("Final selected features: ")
display(X_train)

Fitting estimator with 36 features.
Fitting estimator with 35 features.
Fitting estimator with 34 features.
Fitting estimator with 33 features.
Fitting estimator with 32 features.
Fitting estimator with 31 features.
Fitting estimator with 30 features.
Fitting estimator with 29 features.
Fitting estimator with 28 features.
Fitting estimator with 27 features.
Fitting estimator with 26 features.
Fitting estimator with 25 features.
Fitting estimator with 24 features.
Fitting estimator with 23 features.
Fitting estimator with 22 features.
Fitting estimator with 21 features.
Fitting estimator with 20 features.
Fitting estimator with 19 features.
Fitting estimator with 18 features.
Fitting estimator with 17 features.
Fitting estimator with 16 features.
Fitting estimator with 15 features.
Fitting estimator with 14 features.
Fitting estimator with 13 features.
Fitting estimator with 12 features.
Fitting estimator with 11 features.
Fitting estimator with 10 features.
Fitting estimator with 9 fea

,income,monthly_premium_auto,employmentstatus_Medical Leave,location_code_Suburban,location_code_Urban,vehicle_class_Sports Car,vehicle_size_Medsize,vehicle_size_Small
0,-0.086045,0.345236,-0.26077,-1.056536,1.860011,-0.220641,0.638493,-0.479881
1,0.080159,0.209238,-0.26077,-1.056536,-0.537631,-0.220641,0.638493,-0.479881
2,-1.167396,-1.014741,-0.26077,0.946489,-0.537631,-0.220641,0.638493,-0.479881
3,1.935672,-1.105406,-0.26077,0.946489,-0.537631,-0.220641,-1.566187,2.083851
4,-1.078939,-0.833411,-0.26077,-1.056536,1.860011,-0.220641,-1.566187,-0.479881
...,...,...,...,...,...,...,...,...
4691,-0.235776,-0.969409,-0.26077,0.946489,-0.537631,-0.220641,0.638493,-0.479881
4692,0.399896,1.478550,-0.26077,-1.056536,-0.537631,-0.220641,0.638493,-0.479881
4693,-1.006383,-0.924076,-0.26077,-1.056536,-0.537631,-0.220641,0.638493,-0.479881
4694,0.876518,0.073241,-0.26077,0.946489,-0.537631,-0.220641,0.638493,-0.479881


In [19]:
from sklearn.feature_selection import RFE

#only numerical features

X_train, X_test, y_train, y_test=train_test_split(numericals_features, y, test_size=.20, random_state=42)
std_scaler=StandardScaler().fit(X_train) 

X_train_scaled=std_scaler.transform(X_train)
X_test_scaled=std_scaler.transform(X_test)

model=LinearRegression()
selector = RFE(model, n_features_to_select= 3, step = 1, verbose = 1) # Step is how many features to add or drop everytime
selector.fit(X_train_scaled, y_train)

kept_features = selector.get_support(indices = True) #returns an array of integers corresponding to nonremoved features
kept_features = list(X_train.iloc[:,kept_features].columns)

X_train = selector.transform(X_train_scaled)
X_test  = selector.transform(X_test_scaled)

X_train = pd.DataFrame(X_train, columns=kept_features)
X_test  = pd.DataFrame(X_test, columns=kept_features)

print("Final selected features: ")
display(X_train)


Fitting estimator with 7 features.
Fitting estimator with 6 features.
Fitting estimator with 5 features.
Fitting estimator with 4 features.
Final selected features: 


,income,monthly_premium_auto,months_since_last_claim
0,-0.086045,0.345236,1.496557
1,0.080159,0.209238,0.103232
2,-1.167396,-1.014741,-0.892000
3,1.935672,-1.105406,0.401802
4,-1.078939,-0.833411,-0.195338
...,...,...,...
4691,-0.235776,-0.969409,-0.195338
4692,0.399896,1.478550,0.998941
4693,-1.006383,-0.924076,0.302278
4694,0.876518,0.073241,-0.792477
